In [1]:
%cd ../src

e:\00_CODE\03_Master_Thesis\rdf-literal-preprocessing\src


In [130]:
from dataload import amplus

data = amplus()

loaded data amplus (63.39s).
pruned (37.34s).


In [4]:
POTENTIAL_TEXT_TYPES = [
    '@es', '@fy', '@nl', '@nl-nl', '@pt', '@ru',
    'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString',
    'http://www.w3.org/2001/XMLSchema#string',
    'none'
]

In [137]:
from utils import get_relevant_relations
import re
rr = get_relevant_relations(data,['none'])

In [136]:
data.datatypes()

['iri',
 'blank_node',
 'none',
 'http://kgbench.info/dt#base64Image',
 'http://www.w3.org/2001/XMLSchema#date',
 'http://www.w3.org/2001/XMLSchema#decimal',
 'http://www.w3.org/2001/XMLSchema#positiveInteger']

In [142]:
data.i2e[507071]

('0', 'none')

In [141]:
data.triples[data.triples[:,1]==10]

tensor([[122633,     10, 507071],
        [122638,     10, 507071],
        [122641,     10, 507071],
        ...,
        [506770,     10, 508173],
        [506771,     10, 507071],
        [506773,     10, 507071]], dtype=torch.int32)

In [138]:
rr

[9, 10, 11, 16, 17, 18, 0, 2, 7, 20, 4, 5, 25]

In [132]:
from nltk.corpus import stopwords
import gensim
from gensim.utils import simple_preprocess
import nltk
import torch
#nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim.corpora as corpora
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis
import os
#languages = ['dutch','spain','french','portuguese']
#stopword_list = stopwords.words('dutch')

def get_stopword_list(languages = ['dutch','spanish','french','portuguese','english']):
    stopword_list= []
    for language in languages:
        stopword_list.extend(stopwords.words(language))
    return stopword_list
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(word_array, stopword_list):
    return [word for word in word_array
             if word not in stopword_list]

In [128]:
get_stopword_list()

['de',
 'en',
 'van',
 'ik',
 'te',
 'dat',
 'die',
 'in',
 'een',
 'hij',
 'het',
 'niet',
 'zijn',
 'is',
 'was',
 'op',
 'aan',
 'met',
 'als',
 'voor',
 'had',
 'er',
 'maar',
 'om',
 'hem',
 'dan',
 'zou',
 'of',
 'wat',
 'mijn',
 'men',
 'dit',
 'zo',
 'door',
 'over',
 'ze',
 'zich',
 'bij',
 'ook',
 'tot',
 'je',
 'mij',
 'uit',
 'der',
 'daar',
 'haar',
 'naar',
 'heb',
 'hoe',
 'heeft',
 'hebben',
 'deze',
 'u',
 'want',
 'nog',
 'zal',
 'me',
 'zij',
 'nu',
 'ge',
 'geen',
 'omdat',
 'iets',
 'worden',
 'toch',
 'al',
 'waren',
 'veel',
 'meer',
 'doen',
 'toen',
 'moet',
 'ben',
 'zonder',
 'kan',
 'hun',
 'dus',
 'alles',
 'onder',
 'ja',
 'eens',
 'hier',
 'wie',
 'werd',
 'altijd',
 'doch',
 'wordt',
 'wezen',
 'kunnen',
 'ons',
 'zelf',
 'tegen',
 'na',
 'reeds',
 'wil',
 'kon',
 'niets',
 'uw',
 'iemand',
 'geweest',
 'andere',
 'de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'l

In [117]:
from utils import URI_PREFIX
from preprocess.binning import delete_empty_bin_types
def LDA_topic_assignment(data, num_topics=10, min_mean_word_count = 3, max_assigned_topic = 3, min_topic_relevance = 0.1 ):   
    relevent_relations = get_relevant_relations(
        data, relevant_types=POTENTIAL_TEXT_TYPES)
    #print(num_bins)
    for b in range(num_topics):
        o = (f'{URI_PREFIX}entity#topic{b}', f'{URI_PREFIX}datatype#topics')
        new_id = len(data.i2e)
        data.e2i[o] = new_id
        data.i2e.append(o)
        data.num_entities += 1

    stopword_list = get_stopword_list()
    for r in relevent_relations:
        df = pd.DataFrame(data.triples[data.triples[:,1]== r], columns=['s','p','o'])
        df['text'] = df['o'].apply(lambda t: data.i2e[t][0])
        df['type'] = df['o'].apply(lambda t: data.i2e[t][1])
    
        # delete "none type" polygons 
        df['text'] = df['text'].apply(lambda t:'' if re.match('(MULTIPOLYGON|POLYGON)',t) else t)
        mean_num_words = df['text'].str.count(r'([\w\:\.\/]{3,})').mean()
        #print(mean_num_words)
        if mean_num_words > min_mean_word_count:
            p = f'{URI_PREFIX}predicat#topics{r}'
            new_id = len(data.i2r)
            data.r2i[p] = new_id
            data.i2r.append(p)
            data.num_relations += 1



            df['text_preprocessed'] = df['text'].apply(lambda t: gensim.utils.simple_preprocess(t, deacc=True))
            df['text_preprocessed'] = df['text_preprocessed'].apply(lambda t: remove_stopwords(t, stopword_list=stopword_list))

            data_words = df.text_preprocessed.values.tolist()
            #data_words = list(sent_to_words(text_data))
            print(data_words[:1][0][:30])
            # Create Dictionary
            id2word = corpora.Dictionary(data_words)
            # Term Document Frequency
            corpus = [id2word.doc2bow(text) for text in data_words]
            print(corpus[:1][0][:30])
            num_topics = 10
            lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
            df['vector'] = df['text_preprocessed'].apply(lambda t: id2word.doc2bow(t))
            df['topics'] = df['vector'].apply(lambda t: [x[0] for x in sorted(lda_model.get_document_topics(t), key=lambda x: x[1], reverse=True)[:3] if x[1]>0.1])
            for i in range(num_topics):
                df[f'topic_{i}'] = df['topics'].apply(lambda t: True if i in t else False)
        
            for i in range(num_topics):

                sub = df[df[f'topic_{i}']==True]
                if  len(sub)> 0:
                    sub_df = torch.zeros(len(sub),3,dtype=torch.int32)
                    sub_df[:,0] = torch.tensor(sub.s.tolist(), dtype=torch.int32)
                    #torch.full((len(sub),1),data.r2i[f'{URI_PREFIX}predicat#topics{9}'], dtype=torch.int32)
                    sub_df[:,1] = data.r2i[f'{URI_PREFIX}predicat#topics{r}']
                    sub_df[:,2] = data.e2i[f'{URI_PREFIX}entity#topic{i}', f'{URI_PREFIX}datatype#topics']
                    data.triples = torch.cat((data.triples, sub_df), 0)
    data = delete_empty_bin_types(data,num_topics)
    return data

        #break 

    # remove stop words
    #data_words = remove_stopwords(data_words)

    #print(data_words[:1][0][:30])


    # # Create Dictionary


    # # Create Corpus
    # texts = data_words

    # # Term Document Frequency
    # corpus = [id2word.doc2bow(text) for text in texts]

    # # View
    # print(corpus[:1][0][:30])
    # # number of topics
    # num_topics = 10

    # # Build LDA model
    # lda_model = gensim.models.LdaMulticore(corpus=corpus,
    #                                    id2word=id2word,
    #                                    num_topics=num_topics)

    # # Print the Keyword in the 10 topics
    # print(lda_model.print_topics())
    # doc_lda = lda_model[corpus]
    # models[r] = (lda_model,corpus)

#     # LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
#         t = dfs[9]['text_processed'].iloc[23]
# id2word.doc2bow(remove_stopwords(list(sent_to_words([t])))[0])
# [x[0] for x in sorted(lda_model.get_document_topics(id2word.doc2bow(remove_stopwords(list(sent_to_words([t])))[0])), key=lambda x: x[1], reverse=True)[:3] if x[1]>0.1]
#         break
#         print(r)
#         print(mean_num_words)
    #break

In [114]:
sub_df = torch.zeros(len(sub),3,dtype=torch.int32)

In [115]:
sub_df[:,1]=1

In [116]:
sub_df

tensor([[0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        ...,
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0]], dtype=torch.int32)

In [110]:
torch.full((len(sub),1),data.r2i[f'{URI_PREFIX}predicat#topics{9}'], dtype=torch.int32)

tensor([[61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
        [61],
      

In [129]:
data = LDA_topic_assignment(data)

['rijk', 'gekleed']
[(0, 1), (1, 1)]
['geboren', 'luik', 'overleden', 'amsterdam', 'verbleef', 'jeugd', 'korte', 'tijd', 'rome', 'luik', 'opgeleid', 'bertholet', 'flemalle', 'vestigde', 'voorgoed', 'amsterdam', 'vroegste', 'werk', 'overdadig', 'barok', 'later', 'belargrijkste', 'hollandse', 'classicisme', 'trant', 'schilderde', 'plafonds', 'meestal', 'allegorische', 'mythologische']
[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]
['stadhuis', 'dam', 'amsterdam', 'pieter', 'saenredam']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]


In [123]:
data.i2e[341277]

('http://master-thesis.com/entity#topic7',
 'http://master-thesis.com/datatype#topics')

In [121]:
data.triples

tensor([[130685,     28,  54795],
        [130685,     31, 201822],
        [130690,     28,  58948],
        ...,
        [271522,     61, 341275],
        [271523,     61, 341275],
        [271535,     61, 341277]], dtype=torch.int32)

In [ ]:
def bin_numbers(data: Data, num_bins=3, use_lof=False, num_bins_as_percent=False, equal_height_binning=False, **kwargs):
    relevent_relations = get_relevant_relations(
        data, relevant_types=RDF_NUMBER_TYPES)
    print(num_bins)
    for b in range(num_bins):
        o = (f'{URI_PREFIX}entity#binning{b+1}', f'{URI_PREFIX}datatype#bin')
        new_id = len(data.i2e)
        data.e2i[o] = new_id
        data.i2e.append(o)
        data.num_entities += 1

    for r in relevent_relations:
        p = f'{URI_PREFIX}predicat#binning{r}'
        new_id = len(data.i2r)
        data.r2i[p] = new_id
        data.i2r.append(p)
        data.num_relations += 1

    for relation in relevent_relations:

        sub_df = encode_number_sublist(
            data.triples[data.triples[:, 1] == relation], data.i2e)

        # TODO test new function
        if (use_lof):
            lof = LocalOutlierFactor(n_neighbors=10)
            lof.fit(sub_df[:, 1].reshape(-1, 1))
            outlier_scores = lof.negative_outlier_factor_
            # Create a new column in the numpy array to store the outlier scores
            # tensor_np = torch.hstack((encoded_df, outlier_scores.reshape(-1,1)))
            threshold = np.percentile(outlier_scores, 10)
            # use the outlier scores to filter out the outliers from the numpy array
            sub_df = sub_df[outlier_scores > threshold]

        # numpy is used here since torch.histc was not working for some reason.
        sub_df = torch.cat(  # put bins and sub_df together
            (sub_df, torch.from_numpy(  # get numpy solutions back
                np.digitize(  # assign for each value in sub_df the corresponding bin
                    sub_df[:, 1], np.histogram(  # calculate n bins based on values in sub_df
                        sub_df[:, 1], num_bins)[1][:-1]
                )
            ).reshape(-1, 1)  # transfrom x tensor into (x,1) tensor to fit (x,2) shape of sub_df
            ), 1)

        object_mapping = np.vectorize(lambda t: data.e2i[(
            f'{URI_PREFIX}entity#binning{t}', f'{URI_PREFIX}datatype#bin')])

        predicat_mapping = np.vectorize(
            lambda t: data.r2i[f'{URI_PREFIX}predicat#binning{relation}'])

        sub_df[:, 1] = torch.tensor(np.array([predicat_mapping(sub_df[:, 2])]), dtype=torch.int32)
        sub_df[:, 2] = torch.tensor(np.array([object_mapping(sub_df[:, 2])]), dtype=torch.int32)
        data.triples = torch.cat((data.triples, sub_df), 0)
    data = delete_empty_bin_types(data,num_bins)
    return data

In [73]:
dfs.keys()

dict_keys([9, 57, 53, 39])

In [85]:
dfs[9]

,s,p,o,text,type,text_preprocessed,vector,topics
0,211086,9,28734,"Exterieur, overzicht Administratiegebouw, voor...",@nl-nl,"[exterieur, overzicht, voorgevel, kleine, gebo...","[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...",[3]
1,211090,9,27816,Exterieur overzicht,@nl-nl,"[exterieur, overzicht]","[(1, 1), (6, 1)]",[3]
2,211091,9,27816,Exterieur overzicht,@nl-nl,"[exterieur, overzicht]","[(1, 1), (6, 1)]",[3]
3,211092,9,27816,Exterieur overzicht,@nl-nl,"[exterieur, overzicht]","[(1, 1), (6, 1)]",[3]
4,211093,9,27816,Exterieur overzicht,@nl-nl,"[exterieur, overzicht]","[(1, 1), (6, 1)]",[3]
...,...,...,...,...,...,...,...,...
57037,288864,9,20416,BOERDERIJ. Vrijstaand woonhuis onder pannen sc...,@nl-nl,"[boerderij, vrijstaand, woonhuis, pannen, schi...","[(68, 1), (141, 1), (175, 2), (192, 1), (236, ...",[8]
57038,288866,9,42149,"Ned.Herv.Kerk. Bouwwerk, bestaande uit het vee...",@nl-nl,"[ned, herv, kerk, bouwwerk, bestaande, veelhoe...","[(24, 1), (54, 2), (85, 1), (175, 1), (221, 1)...","[1, 5]"
57039,288868,9,38668,"Korenmolen ""De Koutermolen"". Ronde stenen bove...",@nl-nl,"[korenmolen, koutermolen, ronde, stenen, boven...","[(73, 1), (165, 1), (234, 1), (506, 1), (768, ...","[0, 9]"
57040,288870,9,80531,Woonhuis in gele baksteen met rode bakstenen l...,@nl-nl,"[woonhuis, gele, baksteen, rode, bakstenen, la...","[(5, 1), (42, 2), (175, 1), (192, 1), (236, 1)...",[8]


In [90]:
for i in range(num_topics):
    dfs[39][f'topic_{i}'] = dfs[39]['topics'].apply(lambda t: True if i in t else False)


In [99]:
import torch
sub = dfs[39][dfs[39]['topic_7']==True]
sub_df = torch.zeros(len(sub),3)
sub_df[:,0] = sub.s.tolist()
sub_df[:,1] = data.r2i['key']
sub_df[:,2] = data.e2i['key']
#sub_df = torch.tensor([sub.s.tolist(),len(sub))

In [100]:
sub_df

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [94]:
dfs[39][dfs[39]['topic_7']==True].s.tolist()

[272144,
 272146,
 272148,
 272150,
 272152,
 272154,
 272156,
 272158,
 272160,
 272184,
 272212,
 272218,
 272236,
 272238,
 272240,
 272254,
 272266,
 272268,
 272270,
 272274,
 272312,
 272314,
 272316,
 272318,
 272322,
 272366,
 272370,
 272372,
 272374,
 272442,
 272490,
 272492,
 272494,
 272496,
 272498,
 272500,
 272502,
 272504,
 272506,
 272508,
 272510,
 272512,
 272540,
 272552,
 272570,
 272580,
 272582,
 272608,
 272610,
 272612,
 272614,
 272672,
 272696,
 272718,
 272754,
 272756,
 272758,
 272864,
 272866,
 272868,
 272892,
 272898,
 272906,
 272916,
 272950,
 272974,
 273010,
 273012,
 273014,
 273042,
 273044,
 273046,
 273100,
 273114,
 273168,
 273170,
 273172,
 273174,
 273176,
 273178,
 273180,
 273182,
 273184,
 273186,
 273298,
 273300,
 273302,
 273304,
 273376,
 273378,
 273380,
 273398,
 273402,
 273404,
 273406,
 273408,
 273410,
 273412,
 273414,
 273416,
 273418,
 273420,
 273422,
 273424,
 273458,
 273468,
 273470,
 273472,
 273492,
 273498,
 273518,
 

In [82]:
dfs[39][dfs[39]['topics'].isin([7])] 

,s,p,o,text,type,text_preprocessed,vector,topics


In [48]:
def remove_stopwords(word_array, stopword_list):
    return [word for word in word_array
             if word not in stopword_list]
remove_stopwords(['boerderij', 'vrijstaand', 'woonhuis', 'onder'], get_stopword_list())

['boerderij', 'vrijstaand', 'woonhuis']

In [51]:
df['text_preprocessed']

0        [exterieur, overzicht, voorgevel, kleine, gebo...
1                                   [exterieur, overzicht]
2                                   [exterieur, overzicht]
3                                   [exterieur, overzicht]
4                                   [exterieur, overzicht]
                               ...                        
57037    [boerderij, vrijstaand, woonhuis, pannen, schi...
57038    [ned, herv, kerk, bouwwerk, bestaande, veelhoe...
57039    [korenmolen, koutermolen, ronde, stenen, boven...
57040    [woonhuis, gele, baksteen, rode, bakstenen, la...
57041    [kerk, bonifacius, eenbeukige, neogotische, kr...
Name: text_preprocessed, Length: 57042, dtype: object

In [36]:
df['text'].apply(lambda t: gensim.utils.simple_preprocess(t, deacc=True))

0        [exterieur, overzicht, voorgevel, kleine, gebo...
1                                   [exterieur, overzicht]
2                                   [exterieur, overzicht]
3                                   [exterieur, overzicht]
4                                   [exterieur, overzicht]
                               ...                        
57037    [boerderij, vrijstaand, woonhuis, onder, panne...
57038    [ned, herv, kerk, bouwwerk, bestaande, uit, he...
57039    [korenmolen, de, koutermolen, ronde, stenen, b...
57040    [woonhuis, in, gele, baksteen, met, rode, baks...
57041    [kerk, van, de, bonifacius, eenbeukige, neogot...
Name: text, Length: 57042, dtype: object

In [26]:
dfs = {}

for r in rr:
    df = pd.DataFrame(data.triples[data.triples[:,1]== r], columns=['s','p','o'])
    df['text'] = df['o'].apply(lambda t: data.i2e[t][0])
    df['type'] = df['o'].apply(lambda t: data.i2e[t][1])
    df['text'] = df['text'].apply(lambda t:'' if re.match('(POLYGON)',t) else t)
    mean_num_words = df['text'].str.count(r'([A-Za-z\:\.\/]{3,})').mean()
    #print(mean_num_words)
    if mean_num_words > 3:


        
        df['text_processed'] = df['text'].map(lambda x: '' if re.sub('[\:,\.!?]', '', x))
        df['text_processed'] = df['text_processed'].map(lambda x: x.lower())
        # for word in dutch_stopwords:
        #     df['text_processed'] = df['text_processed'].map(lambda x: re.sub(rf'[][{word}][]', '', x)) 
        # df['text_processed'] = df['text_processed']

        dfs[r] = df

        # Join the different processed titles together.
        long_string = ' '.join(list(df['text_processed'].values))

        #for word in dutch_stopwords:
        #    long_string= long_string.replace(f'{word} ','')
        #long_string= long_string.replace(f'newline ','')

        # Create a WordCloud object
        wordcloud = WordCloud(background_color="white", max_words=1000, contour_width=3, contour_color='steelblue')

        # Generate a word cloud
        wordcloud.generate(long_string)

        # Visualize the word cloud
        wordcloud.to_image().show()




SyntaxError: invalid syntax (750369112.py, line 14)

In [178]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim.corpora as corpora
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis
import os

stop_words = stopwords.words('dutch')
stop_words.extend(['newline'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
models = {}
for r,df in dfs.items():
    text_data = df.text_processed.values.tolist()
    data_words = list(sent_to_words(text_data))

    # remove stop words
    data_words = remove_stopwords(data_words)

    print(data_words[:1][0][:30])


    # Create Dictionary
    id2word = corpora.Dictionary(data_words)

    # Create Corpus
    texts = data_words

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]

    # View
    print(corpus[:1][0][:30])
    # number of topics
    num_topics = 10

    # Build LDA model
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

    # Print the Keyword in the 10 topics
    print(lda_model.print_topics())
    doc_lda = lda_model[corpus]
    models[r] = (lda_model,corpus)

    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    # # Visualize the topics
    # pyLDAvis.enable_notebook()

    # LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))

    # # # this is a bit time consuming - make the if statement True
    # # # if you want to execute visualization prep yourself
    # if 1 == 1:
    #     LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    #     with open(LDAvis_data_filepath, 'wb') as f:
    #         pickle.dump(LDAvis_prepared, f)

    # # load the pre-prepared pyLDAvis data from disk
    # with open(LDAvis_data_filepath, 'rb') as f:
    #     LDAvis_prepared = pickle.load(f)

    # pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')

    # LDAvis_prepared
    break

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Noctris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['exterieur', 'overzicht', 'voorgevel', 'kleine', 'gebouw', 'links', 'midden', 'wachtkamer', 'apotheek']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]
[(0, '0.042*"zijgevel" + 0.018*"overzicht" + 0.015*"voorgevel" + 0.010*"rechter" + 0.008*"vensters" + 0.006*"twee" + 0.005*"boven" + 0.005*"deel" + 0.005*"achtergevel" + 0.004*"exterieur"'), (1, '0.045*"voorgevel" + 0.018*"nr" + 0.013*"achtergevel" + 0.013*"zijgevel" + 0.006*"overzicht" + 0.005*"houten" + 0.005*"vensters" + 0.005*"kerk" + 0.004*"voorzien" + 0.004*"twee"'), (2, '0.014*"achtergevel" + 0.011*"nr" + 0.011*"achtkante" + 0.010*"voorgevel" + 0.010*"overzicht" + 0.009*"gefotografeerd" + 0.008*"aanzicht" + 0.007*"twee" + 0.007*"gevel" + 0.006*"exterieur"'), (3, '0.024*"exterieur" + 0.011*"twee" + 0.008*"zijgevel" + 0.006*"achtkante" + 0.006*"vensters" + 0.006*"gevel" + 0.005*"achtergevel" + 0.004*"belang" + 0.004*"houten" + 0.004*"vanwege"'), (4, '0.063*"voorgevel" + 0.030*"overzicht" + 0.022*"exterieur

c:\Users\Noctris\.virtualenvs\rdf-literal-preprocessing-20b3_M0v\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [202]:
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]


In [233]:
t = dfs[9]['text_processed'].iloc[23]
id2word.doc2bow(remove_stopwords(list(sent_to_words([t])))[0])
[x[0] for x in sorted(lda_model.get_document_topics(id2word.doc2bow(remove_stopwords(list(sent_to_words([t])))[0])), key=lambda x: x[1], reverse=True)[:3] if x[1]>0.1]

# remove_stopwords(t)
#     data_words = list(sent_to_words(text_data))

#     # remove stop words
#     data_words = remove_stopwords(data_words)

[9]

In [217]:
dfs[9]['text_processed'].iloc[0]

'exterieur overzicht administratiegebouw voorgevel binnengasthuisstraat 9  kleine gebouw links/midden is wachtkamer/apotheek'

In [201]:
len(id2word.keys())

43015

In [210]:
lda_model.show_topic(9)

[('overzicht', 0.040573794),
 ('exterieur', 0.028770167),
 ('gevel', 0.015426227),
 ('voorgevel', 0.01178268),
 ('eeuw', 0.008472809),
 ('boerderij', 0.006760539),
 ('zuidgevel', 0.005024846),
 ('twee', 0.0049154814),
 ('toren', 0.0045687947),
 ('voorzijde', 0.0043822383)]

In [200]:
lda_model.top_topics(corpus)[0]

([(0.042350933, 'zijgevel'),
  (0.017903773, 'overzicht'),
  (0.014605292, 'voorgevel'),
  (0.010351551, 'rechter'),
  (0.007596633, 'vensters'),
  (0.006375451, 'twee'),
  (0.005431587, 'boven'),
  (0.005430931, 'deel'),
  (0.0054016714, 'achtergevel'),
  (0.004311797, 'exterieur'),
  (0.0039438996, 'houten'),
  (0.003926319, 'huis'),
  (0.0038003884, 'verdieping'),
  (0.0035305512, 'linker'),
  (0.0034399512, 'noordgevel'),
  (0.0034246587, 'deur'),
  (0.003343821, 'woongedeelte'),
  (0.0032433395, 'drie'),
  (0.0029321224, 'xviii'),
  (0.0029315404, 'vanwege')],
 -2.614338742936087)

In [130]:
        # Import the wordcloud library
from wordcloud import WordCloud
for r in rr:
    df = pd.DataFrame(data.triples[data.triples[:,1]== r], columns=['s','p','o'])
    df['text'] = df['o'].apply(lambda t: data.i2e[t][0])
    df['type'] = df['o'].apply(lambda t: data.i2e[t][1])
    mean_num_words = df['text'].str.count(r'([A-Za-z\:\.\/]{3,})').mean()
    print(mean_num_words)
    #if mean_num_words > 3:
    #print("hi")

        #df['text_processed'] = df['text'].map(lambda x: re.sub('[\:,\.!?]', '', x))
        #df['text_processed'] = df['paper_text_processed'].map(lambda x: x.lower())


        # # Join the different processed titles together.
        # long_string = ','.join(list(df['paper_text_processed'].values))

        # # Create a WordCloud object
        # wordcloud = WordCloud(background_color="white", max_words=1000, contour_width=3, contour_color='steelblue')

        # # Generate a word cloud
        # wordcloud.generate(long_string)

        # # Visualize the word cloud
        # wordcloud.to_image()

## Print out the first rows of papers
#papers['paper_text_processed'].head()

1.0079872204472844
0.0
1.1012987012987012
2.0
0.9636363636363636
0.0
1.2016129032258065
0.0
6.784874253663831e-05
2.2054044468089824
23.614512113881
2.005254938375823
2.205349831680466
2.6569912519718915
2.242540904716073
2.0
1.8888888888888888
3.5714285714285716
2.0
1.1748451643639828
0.0
2.4165078608861363
0.0
0.00023820867079561695
1.5560371517027864
0.009433962264150943
0.005482456140350877
1.0
0.0
0.0
1.641025641025641
1.0103761348897535
1.8713826366559485
1.1085271317829457
1.0


In [ ]:
    df = pd.DataFrame(data.triples[data.triples[:,1]== 53], columns=['s','p','o'])
    df['text'] = df['o'].apply(lambda t: data.i2e[t][0])
#df['text'] = df['text'].astype('str') 
    df['type'] = df['o'].apply(lambda t: data.i2e[t][1])

In [ ]:
df

,s,p,o,text,type
0,271513,53,25961,"Dusselsma, N.W.",none
1,271514,53,23730,"Cuypers, Pierre Joseph Hubert",none
2,271515,53,23730,"Cuypers, Pierre Joseph Hubert",none
3,271522,53,23730,"Cuypers, Pierre Joseph Hubert",none
4,271523,53,23730,"Cuypers, Pierre Joseph Hubert",none
5,271524,53,74522,"Sandt, Anton van der",none
6,271535,53,26360,"Embden, F.C.E. van",none


In [65]:
import pandas as pd
df = pd.DataFrame(data.triples[data.triples[:,1]== 32], columns=['s','p','o'])
df['text'] = df['o'].apply(lambda t: data.i2e[t][0])
#df['text'] = df['text'].astype('str') 
df['type'] = df['o'].apply(lambda t: data.i2e[t][1])
#df['count_words'] = df['text'].str.match(r'[A-Za-z0-9]+').str.get(0).groupby(lambda x: x).count()

In [49]:
df['text'] = df['text'].dtype

In [67]:
df['text'].str.count(r'([A-Za-z0-9]+)').mean()

2.8433873376996437

In [35]:
data.i2r[17]

'http://purl.org/collections/nl/am/exhibitionTitle'

In [34]:
df[df['type']=='none']

,s,p,o,text,type
0,122694,17,53008,Rijk gekleed 1750-1914,none
1,122727,17,66314,Zomertentoonstelling,none
2,122766,17,35585,Goed Verkeerd. Postertentoonstelling,none
3,122814,17,65429,Willem Roelofs (1822-1897),none
4,122849,17,21992,Bergen aan Zee 100 jaar,none
...,...,...,...,...,...
10466,506593,17,37219,Het geschenk. De Hollandse Meesters van een Am...,none
10467,506620,17,45503,Max Liebermann en zijn Nederlandse kunstenaars...,none
10468,506664,17,35587,Goed verkeerd. Geschiedenis van homoseksuele m...,none
10469,506695,17,32051,Een huis vol bloemen : zomerse kleurenpracht i...,none


In [19]:
df['text'] = df['o'].apply(lambda t: data.i2e[t])
df['type'] = df['o'].apply(lambda t: data.i2e[t][1])

In [21]:
df[df['type']=='none']

,s,p,o,text,type
1890,127442,9,508236,"(hoogte uitgeklapt, none)",none
59336,270079,9,507103,"(diepte uitgeklapt, none)",none
